In [1]:
import pandas as pd
import os
from os import listdir, rename
from os.path import isfile, join, exists
from shutil import copyfile

Put the path to the folder holding the images in a variable.
Then import the Excel spreadsheet using Pandas.
Convert the ISBN column to string, otherwise it's a 13 digit integer.
Once we have the Excel data, grab the two columns we want and put them in a DataFrame. And drop empty rows.

In [2]:
img_path = r'D:\Coding\Python\jupyter\Cards'

In [3]:
data = pd.read_excel(r'D:\Coding\Python\jupyter\Cards\docs\_CatalogMaster.xlsx', dtype={'ISBN': str})

In [4]:
df = pd.DataFrame(data, columns=['ISBN', 'ProductCode']).dropna(how='all')

Go get a list of all the image files on the drive. And make sure it's only files and not directories.

In [5]:
img_files_list = [f for f in listdir(img_path) if isfile(join(img_path, f))]

Then make a function we can use to lookup data in the DataFrame without stopping each time it doesn't find one.

In [6]:
def try_get_isbn(product_code):
    try:
        line = df.loc[df['ProductCode'] == product_code]
        isbn = (True, line.iloc[0,0])
    except:
        isbn = (False, 'Product code not found')  
    return isbn

The main event. Pulls the product code out of the file name, builds a list of matching files, and then renames them according to the specified naming scheme. Also writes all activity to a log file.

In [10]:
with open(join(img_path, "log.txt"), "a") as log_file:
    for image in img_files_list:
        img_product_code = image.split("-")[0]
        img_sub_list = [f for f in listdir(img_path) if img_product_code == f.split("-")[0]]

        for img in img_sub_list:
            isbn = try_get_isbn(img_product_code)
            if isbn[0]:
                if exists(join(img_path, f"{isbn[1]}.jpg")):
                    rename(join(img_path, img), join(img_path, f"{isbn[1]}_1.jpg"))
                    log_file.write(f"{img}, {isbn[1]}_1.jpg\n")
                else:
                    rename(join(img_path, img), join(img_path, f"{isbn[1]}.jpg"))
                    log_file.write(f"{img}, {isbn[1]}.jpg\n")
            else:
                log_file.write(f"{img}, {isbn[1]}\n")
